## Multivariate Modeling - Exogenous

I will first expand on the simple univariate ARIMA models by adding in other predictors as *exogenous* variables, meaning that I will treat the influence of economic and demographic factors as uni-directional. 

This will provide a simpler model, and allow us to assess the degree of influence of each of these factors.

#### Importing Packages

In [1]:
import pandas as pd
import numpy as np
import requests
import json
import os
import time
import fred_msa
import ts_functions
import datetime as dt
from datetime import date
import regex as re
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter

from sklearn.model_selection import TimeSeriesSplit
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import pacf
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.arima.model import ARIMA

import warnings
warnings.filterwarnings('ignore')

#### Ingesting Data

In [2]:
hpi = pd.read_csv('..\\working-data\\hpi-data.csv')
hpi = hpi.query("year <= 2019")
hpi.date = pd.to_datetime(hpi.date)
#hpi = hpi.set_index(['city', 'msa_state', 'date'])

In [16]:
best_hypers = pd.read_csv('../model-results/best_hypers.csv')

In [56]:
hpi[endog_cols].diff(3)

,hpi,weekly_wages,rpp,population,gdp,unemployment_rate,crimes,subprime,poverty,patents,private_establishments,premature_death_rate,snap
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-20.065172,-1.896431,0.06750,8.99775,-254.07825,2.2,-73.500000,-0.369389,1.113981,-2.500000,403.0,-6.572641,12008.25
4,-11.341184,4.286556,0.06750,8.99775,-254.07825,3.1,-73.500000,-0.257918,1.113981,-2.333333,-914.0,-6.572641,12008.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...
955,12.266284,10.308779,0.24375,-42.69450,60980.56950,-0.6,-185.000000,0.143451,-0.494449,-117.218182,8523.0,1.664579,-66291.00
956,11.703038,29.670534,0.24375,-42.69450,60980.56950,-0.3,-185.000000,0.165771,-0.494449,139.022727,-4357.0,1.664579,-66291.00
957,7.143122,43.087747,0.16250,-28.46300,40653.71300,-0.2,-123.333333,-0.388124,-0.329633,497.712121,7951.0,1.109719,-44194.00
958,6.650333,48.024484,0.08125,-14.23150,20326.85650,-0.2,-61.666667,-0.655868,-0.164816,-289.181818,7482.0,0.554860,-22097.00


In [4]:
def ARIMA_pred(data, city, order=(2,2,0), trend='n'):
    
    # get data from one city and convert to array
    city_data = data[data.city == city].reset_index(drop=True)
    X = np.array(city_data.hpi)
    
    # set up df to store results
    results_df = city_data.set_index('date')['hpi'].to_frame()
    results_df = results_df.assign(pred_1=np.nan, pred_2=np.nan, pred_3=np.nan, pred_4=np.nan)
    
    # run predictions
    tscv = TimeSeriesSplit(n_splits=36, max_train_size=12, test_size=1)
    
    for train_index, test_index in tscv.split(X):
        X_train = X[train_index] # training set
        date = city_data.date[test_index[0]] # find date of first prediction
        city_model = ARIMA(X_train, order=order, trend=trend).fit() # train model
        preds = city_model.get_forecast(4) # predict 4 periods ahead
        pred_means = preds.predicted_mean
        pred_stds = preds.se_mean
    
    return preds, preds_mean, preds_std, city_model.aic

In [11]:
exog_cols = ['weekly_wages', 'rpp', 'population', 'gdp', 'unemployment_rate', 'crimes', 'subprime', 'poverty', 'patents', 'private_establishments', 'premature_death_rate', 'snap']

In [95]:
endog_cols = ['hpi', 'weekly_wages', 'rpp', 'population', 'gdp', 'unemployment_rate', 'crimes', 'subprime', 'poverty', 'patents', 'private_establishments', 'premature_death_rate', 'snap']
exog_cols = ['weekly_wages', 'rpp', 'gdp']

In [17]:
best_hypers.query("city=='New York'")

,city,p,d,q
13,New York,2,2,0


In [18]:
data=hpi
city='New York'
order=(2,2,0)
trend='n'

In [96]:
# get data from one city and convert to array
city_data = data[data.city == city].reset_index(drop=True)
endog = np.array(city_data.hpi)
exog = np.array((city_data[exog_cols] - city_data[exog_cols].mean())/city_data[exog_cols].std())


# run predictions
tscv = TimeSeriesSplit(n_splits=36, max_train_size=12, test_size=1)

In [100]:
hpi

,date,quarter,msa_state,year,city,hpi,weekly_wages,rpp,population,gdp,unemployment_rate,crimes,subprime,poverty,patents,private_establishments,premature_death_rate,snap
0,2008-01-01,1,SC,2008,Charleston,361.226779,733.576515,97.64100,647.19400,2.727761e+04,4.8,10083.000000,42.546860,13.983937,7.000000,16277.0,395.327521,69915.00
1,2008-04-01,2,SC,2008,Charleston,355.616583,728.179263,97.66350,650.19325,2.719292e+04,5.1,10058.500000,42.203499,14.355264,6.000000,16760.0,393.136641,73917.75
2,2008-07-01,3,SC,2008,Charleston,344.436457,732.479714,97.68600,653.19250,2.710823e+04,6.0,10034.000000,42.138782,14.726591,4.000000,16960.0,390.945760,77920.50
3,2008-10-01,4,SC,2008,Charleston,341.161607,731.680084,97.70850,656.19175,2.702353e+04,7.0,10009.500000,42.177472,15.097918,4.500000,16680.0,388.754880,81923.25
4,2009-01-01,1,SC,2009,Charleston,344.275399,732.465819,97.73100,659.19100,2.693884e+04,8.2,9985.000000,41.945581,15.469245,3.666667,15846.0,386.564000,85926.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
955,2018-10-01,4,NY,2018,New York,383.538057,1504.986268,114.63475,19246.72550,1.851839e+06,3.7,28068.666667,23.718218,11.821278,2959.681818,627302.0,318.105794,2344509.00
956,2019-01-01,1,NY,2019,New York,388.377299,1527.067602,114.71600,19232.49400,1.872166e+06,3.8,28007.000000,23.224716,11.656461,2942.750000,616838.0,318.660653,2322412.00
957,2019-04-01,2,NY,2019,New York,387.369467,1546.876599,114.71600,19232.49400,1.872166e+06,3.6,28007.000000,22.828048,11.656461,3196.000000,630807.0,318.660653,2322412.00
958,2019-07-01,3,NY,2019,New York,390.188390,1553.010752,114.71600,19232.49400,1.872166e+06,3.5,28007.000000,23.062351,11.656461,2670.500000,634784.0,318.660653,2322412.00


In [98]:
i = 0
for train_index, test_index in tscv.split(endog):
    endog_train = endog[train_index] # training set
    exog_train = exog[train_index]
    date = city_data.date[test_index[0]] # find date of first prediction
    city_model = ARIMA(endog_train, exog=exog_train, order=order, trend=trend).fit() # train model
    #print(i)
    i+=1
    preds = city_model.get_forecast(4, exog=np.broadcast_to(exog_train[-1], (4, 3))) # predict 4 periods ahead
    pred_means = preds.predicted_mean
    pred_stds = preds.se_mean

In [99]:
city_model.aic

47.6379778697591

In [45]:
pred_means

array([7.99824269e+11, 1.59964854e+12, 2.39947281e+12, 3.19929707e+12])

In [46]:
pred_stds

array([ 2.1096162 ,  4.71725476,  7.89349237, 11.55490412])

In [73]:
endog_train.shape

(12, 13)